# <center> Locality Sensitive Hashing </center>
## <center> Brutal force approach </center>
## <center> 2$^{nd}$ Assignment </center>
### <center> By Group 80 for the course Advances in Data Mining of the University of Leiden taught by Wojtek Kowalczyk </center>
<center> Lisa Dombrovskij (s1504819) - dombrovskij@strw.leidenuniv.nl </center>
<center> Margherita Grespan (s2233150) - grespan@strw.leidenuniv.nl </center>

***

The [Netflix challenge](https://www.netflixprize.com/) was an open competition held by Netflix, an online movie rental platform. The competition was started in order to find the best collabrative filtering algorithm. This algorithm asked to predict the ratings of a user for a specific movie based on previous ratings.

In this notebook we show the most basic - and most time consuming - approach to calculate the similarities between all the users in the dataset. In this dataset two users can be defined similar if they have watched a similar set of movies. The similarities are calculated using the Jaccard similarity. All user pairs with a Jaccard similarity above a certain threshold are defined as similar.

## The dataset - Netflix Challenge
Netflix, for the Netflix challenge, provided a dataset with 100.480.507 ratings given by 480.189 users to 17.770 movies. 
In this notebook we use an extract of the Netflix dataset that contains:
-  about 65.225.506 combinations of a user and movie
-  103703 users that rated a total of 17.770 different movies;
-  Each user rated between 300 and 3000 movies

The raw dataset consists in a list of 65.000.000 records, each record consists of two integers: *user_id* and *movie_id*.

## The brute force approach

If two users have watched similar movies, it is more likely that they will both like the same movies in the future. If $user_1$ liked a movie, it is straightforward to think that $user_2$ - who has the same taste in movies - will like that movie too. Based on this idea it is useful to find the similarity between each user pair, to find users who are similar. 
The easiest way to find if two users are similar is to compare them; Basically, check if $user_1$ watched what $user_2$ watched. 

This process seems simple, but with big datasets it is not doable in terms of time. In this work we will try to estimate the time needed to calculate the similarity between all the possible user pairs in the dataset.
The order of a pair does not matter, 
<center>$(user_1, user_2) = (user_2, user_1)$</center>

Knowing this, the amount of pairs in the datasest is computable using the equation for combinations without permutations:

<center> $C(n,k)=\frac{n!}{(n-k)!k!}$</center>

where n is the amount of users in the dataset and k is 2.

In [7]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.model_selection import KFold
import itertools
from random import sample
import random
np.warnings.filterwarnings('ignore')

First the data is loaded. It consists of about 65.000.000 records of user-movie combinations. The data is converted to a binary table with 103703 rows (the amount of unique users in the set) and 17770 columns (the amount of unique movies in the dataset). When a user has watched a movie, the value in the corresponding row is 1, otherwise it is 0.

In [2]:
data=np.load('./user_movie.npy')
n_users=len(np.unique(data[:,0]))
print(' **** Information about the dataset ****')
print('Amount of users:', n_users)
print('Amount of movies:',len(np.unique(data[:,1])))
n_tot_comb=np.math.factorial(n_users)/(np.math.factorial(n_users-2)*np.math.factorial(2))
print('Total amount of possible combinations:',n_tot_comb)
print(' ')
print('**** What the dataset looks like ****')

#data stored in a dataframe
data = pd.DataFrame(data, columns = ['ID', 'Movie']) 
#In our dataset we don't have the movie ratings, so we create a binary dataframe. 
#The movie has been watched/rated -> 1
#The movie hasn't been rated -> 0
df_users_movie = np.zeros((data.ID.nunique(),data.Movie.nunique()))
df_users_movie[data.ID.values, data.Movie.values] = 1
df_users_movie = pd.DataFrame(df_users_movie) 
df_users_movie.head()

 **** Information about the dataset ****
Amount of users: 103703
Amount of movies: 17770
Total amount of possible combinations: 5377104253.0
 
**** What the dataset looks like ****


,0,1,2,3,4,5,6,7,8,9,...,17760,17761,17762,17763,17764,17765,17766,17767,17768,17769
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Jaccard Similarity

The goal of this project is to find similar users, i.e. users that have watched a similar set of movies. The similarity measure that will be used is called the Jaccard similarity. It measures the similarity between finite sets of datapoints, and is defined as the size of the intersection divided by the size of the union of the sets. The equation looks as follows:

$J(A,B) =  \frac{|A \cap B|}{|A \cup B|}$

where A and B are the two collections of datapoints. 
It returns a number between 0 and 1, that reflects how similar the two sets are. We define two users to be similar if their Jaccard similarity is higher than 0.5.

In [3]:
#Function to calculate the Jaccard similarity
def jsim(A, B):
    return sum(A*B) / np.count_nonzero(A+B)

In [4]:
def draw_pair():
    
    '''
    Returns a random pair of users that is not the same user.
    '''
    
    a = int(random.uniform(0,df_users_movie.shape[1]))
    b = int(random.uniform(0,df_users_movie.shape[1]))
    
    while a == b:
        a = int(random.uniform(0,df_users_movie.shape[1]))
        b = int(random.uniform(0,df_users_movie.shape[1]))
    
    return (a,b)

In [5]:
def create_pair_list(n_pairs):
    
    '''
    Creates a list of n_pairs pairs of users without doubles.
    '''
    
    pair_list = []
    
    for k in range(n_pairs):
    
        pair = draw_pair()

        while pair in pair_list:
            pair = draw_pair()

        pair_list.append(pair)
    
    return pair_list

In [8]:
pair_list = create_pair_list(100)

## Estimating the Total Time

We estimate the total time by taking a specific amount of random pairs of users and measuring how long it takes to calculate all of the Jaccard similarities for these pairs. This time is then exterpolated to the total amount of time it will take for all pairs in the dataset by using the total amount of combinations. The equation becomes:

$TotalTime = \frac{SubsetTime*TotalComb}{SubsetComb}$

with TotalTime the total time it takes to calculate the Jaccard similarity for each pair in the dataset, SubsetTime the time it takes to calculate the Jaccard similarities for the subset of random pairs, TotalComb the total amount of combinations in the dataset and SubsetComb the total amount of combinations in the subset. 

We take 100 random pairs for the first subset and calculate the TotalTime using the above equation. We then do this again for 500, 1000, 1200, 1500 and 2000 pairs. For each subset we calculate the TotalTime. The final estimation of the TotalTime is the mean of all of the TotalTime's of the subsets. 

In [9]:
n_tot_comb=np.math.factorial(n_users)/(np.math.factorial(n_users-2)*np.math.factorial(2))
print('The total amount of comibination with ',n_users,'users is ', n_tot_comb)

The total amount of comibination with  103703 users is  5377104253.0


In [10]:
start_time = time.time()

sim_list = np.zeros(len(pair_list))

for i, comb in enumerate(pair_list):
    
    a = df_users_movie.values[:,comb[0]]
    b = df_users_movie.values[:,comb[1]]
    
    sim_list[i] = jsim(a,b)
    
elapsed_time = time.time() - start_time
total_t= (elapsed_time/len(pair_list)) * n_tot_comb


print('Total time to run the model for ',str(len(pair_list)),'combinations: {}'.format(round(elapsed_time,2)),'s')
print('Total time to run the model for all the',str(int(n_tot_comb)),'combinations :',str(round(total_t,2)),'s')
print('in hours:', str(round(total_t/(3600*24),2)))


Total time to run the model for  100 combinations: 60.28 s
Total time to run the model for all the 5377104253 combinations : 3241072708.11 s
in hours: 37512.42
